In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go 

### Reading In Tables
    Keys built in

In [96]:
admin_df = pd.read_csv('../Data/Mart2/admin.csv')
other_df = pd.read_csv('../Data/Mart2/other.csv')
locat_df = pd.read_csv('../Data/Mart2/location_compiled.csv')
admin_df.drop('Unnamed: 0', axis=1, inplace=True)
other_df.drop('Unnamed: 0', axis=1, inplace=True)
locat_df.drop('Unnamed: 0', axis=1, inplace=True)

# dropping duplicates throughout
admin_df.drop_duplicates('Incident_Key', inplace=True) # minimal loss
other_df.drop_duplicates('Incident_Key', inplace=True) # minimal loss
locat_df.drop_duplicates(inplace=True) # duplicated Incident_Key include multiple primary locations

In [127]:
# census data for 2019 population estimates, source:
# https://www2.census.gov/programs-surveys/popest/tables/2010-2019/counties/totals/
census = pd.read_excel('../Data/Mart2/tn_county_pop.xlsx',sheet_name='Sheet1')

In [103]:
census.head()

,County,2019 Estimate
0,Anderson,76978
1,Bedford,49713
2,Benton,16160
3,Bledsoe,15064
4,Blount,133088


In [66]:
# missing city
admin_df.Address.isnull().value_counts()

False    59056
True       918
Name: Address, dtype: int64

### Decided to run county for time
Other includes county without needing to parce inconsistent/broken address field from Admin

In [99]:
def mult_pri_local(key, list_of_locations):
    # number of times to duplicate entries, but not the weights (for Rates)
    n = len(list_of_locations)
    for i in range(n):
        mart2_dict['Incident_Key'].append(key)
        mart2_dict['Incident_Date'].append(dt)
        mart2_dict['County'].append(county)
        if i == 0:
            mart2_dict['Weight'].append(weight)
        else:
            mart2_dict['Weight'].append(0)
        mart2_dict['Agency_Name'].append(agency_name)
        mart2_dict['Primary_Location'].append(list_of_locations[i])

    return

mart2_dict = {
    'Incident_Key':list(admin_df.Incident_Key),
    'Incident_Date':[],
    'County':[],
    'Weight':[],
    'Agency_Name':[],
    'Primary_Location':[],
}

for key in list(admin_df.Incident_Key):
    oth = other_df.loc[other_df.Incident_Key == key]
    # pull county from other_df
    county = oth.County.values[0]
    # pull weight
    weight = oth['Number of Offenses in Incident'].values[0]
    # pull Agency Name
    agency_name = oth['Agency Name'].values[0]
    locat = locat_df.loc[locat_df.Incident_Key == key]
    # pull date
    dt = locat['Incident Date'].values[0]
    # pull Pri Local(s)
    pri_loc = locat['Primary Location Type'].unique().tolist()
    
    # append to dictionary
    mult_pri_local(key,pri_loc)

print(i)


160


In [138]:
locat_df.loc[locat_df.Incident_Key == 'TN0410000<>20010101-133<>01/01/20']['Primary Location Type'].unique().tolist()

['Residence/Home']

In [104]:
# creating table
mart2_df = pd.DataFrame(mart2_dict)
mart2_df.head()

,Incident_Key,Incident_Date,County,Weight,Agency_Name,Primary_Location
0,TN0410000<>20010101-133<>01/01/20,01/01/20,Hickman,1,Hickman County Sheriff's Office,Residence/Home
1,TN0150100<>202001000000<>01/02/20,01/02/20,Cocke,1,Newport Police Department,Residence/Home
2,TN0410000<>200103-2142<>01/03/20,01/03/20,Hickman,1,Hickman County Sheriff's Office,Residence/Home
3,TN0500100<>20010166<>01/02/20,01/02/20,Lawrence,1,Lawrenceburg Police Department,Parking Lot/Garage
4,TN0540300<>2020010076<>01/01/20,01/01/20,McMinn,1,Etowah Police Department,Residence/Home


In [133]:
by_county = mart2_df.groupby(['County']).sum()
by_county.reset_index(drop=False, inplace=True)
by_county.rename({'Weight':'DV_Freqeuncy'}, axis=1, inplace=True)
by_county['Pop_Est_2019'] = census['2019 Estimate']
by_county['Rate Per 1k'] = round(by_county.DV_Freqeuncy / by_county.Pop_Est_2019 * 1000, 4)
by_county['Rate Per 10k'] = round(by_county.DV_Freqeuncy / by_county.Pop_Est_2019 * 10000, 4)
by_county['Rate Per 100k'] = round(by_county.DV_Freqeuncy / by_county.Pop_Est_2019 * 100000, 4)
by_county.head()

,County,DV_Freqeuncy,Pop_Est_2019,Rate Per 1k,Rate Per 10k,Rate Per 100k
0,Anderson,681,76978,8.8467,88.4668,884.6683
1,Bedford,469,49713,9.4342,94.3415,943.4152
2,Benton,32,16160,1.9802,19.8020,198.0198
3,Bledsoe,16,15064,1.0621,10.6213,106.2135
4,Blount,885,133088,6.6497,66.4974,664.9736


In [128]:
# # Checking matching index
# for i in range(len(by_county)):
#     bc_county = by_county.iloc[i].County
#     c_county = census.iloc[i].County
#     if bc_county != c_county:
#         print(i, bc_county, c_county)

In [98]:
', '.join(locat_df.loc[locat_df.Incident_Key == 'TN0790200<>202000000000<>01/24/20']['Primary Location Type'].unique().tolist())

'Hospital, Residence/Home'

In [ ]:
# Features: 
# -City, ignored for time, might try a partial 
# -County, other
# -Agency Name, other
# -Pri Location, locat
# -Pop of City / County, investigate this...
# -Rates per 1k, 10k, 100k residents
# -Weigth (essentially how many offenses were reported in this incident), other

In [35]:
# hunting for City
broken = [] # includes missing and broken addresses
for i in range(len(admin_df)):
    row = admin_df.iloc[i]
    st = row.Address
    # checking if 'TN' is second to last entry on all of them
    try:
        check = st.split(',')[-2]
    except (AttributeError, IndexError):
        key = row.Incident_Key
        broken.append(key)
        continue

    if 'TN' not in check:
        key = row.Incident_Key
        broken.append(key)

len(broken)

2057

In [ ]:
# 369 have ",,,,," as address
# 918 isnull() is True
# 47 Can have cities/Towns extracted manually
i = 0
for key in broken:
    X = admin_df.loc[admin_df.Incident_Key == key]
    if X.Address.values[0] != ',,,,,' and type(X.Address.values[0]) == str: 
        try:
            place = X.Address.values[0].split(',')[-3]
            if place != '':
                i+=1 
                print(key, place)
        except IndexError:
            continue
print(i)

key_city = {
    'TN0010000<>2001100069<>01/10/20': 'POWELL',
    'TN0380000<>2020-000796<>01/18/20': 'BROWNSVILLE',
    'TN0510000<>SH20032026<>01/23/20': 'HOHENWALD',
    'TN0600100<>2020-000968<>02/21/20':'COLUMBIA',
    'TN0600100<>2020-001108<>02/27/20': 'COLUMBIA',
    'TN0740100<>2020-000316<>02/10/20': 'SPRINGFIELD',
    'TN0470000<>2001030154<>01/03/20': 'KNOXVILLE',
    'TN0610000<>202002499<>03/31/20': 'TEN MILE',
    'TN0380000<>2020-003189<>03/09/20': 'BROWNSVILLE',
    'TN0600100<>2020-001258<>03/05/20': 'COLUMBIA',
    'TN0470000<>2002291923<>02/23/20': 'HEISKELL',
    'TN0470000<>2002291940<>02/29/20': 'POWELL',
    'TN0470000<>2004060197<>04/06/20': 'MASCOT',
    TN0470000<>2004080311<>04/08/20 POWELL
    TN0440000<>030320-13288<>03/03/20 GAINESBORO
    TN0470000<>2002281885<>02/28/20 POWELL
    TN0470000<>2004130549<>04/13/20 KNOXVILLE
    TN0740000<>2020-001070<>04/10/20 CEDAR HILL
    TN0410000<>200503-1652<>05/03/20 LYLES
    TN0440000<>052120-13385<>05/21/20 GAINESBORO
    TN0950100<>2020025898<>06/24/20 Lebanon
    TN0440000<>060820-13406<>06/08/20 GAINESBORO
    TN0290300<>2006280309<>06/28/20 BEAN STATION
    TN0290300<>2007110171<>07/11/20 BEAN STATION
    TN0470000<>2007060390<>07/06/20 STRAWBERRY PLAINS
    TN0470000<>2007120722<>07/12/20 KNOX
    TN0470000<>2007120717<>07/12/20 CORRYTON
    TN0470000<>2006100679<>06/10/20 KNOX
    TN0470000<>2007181112<>07/18/20 KNOX
    TN0860000<>2007002253<>07/26/20 UNICOI
    TN0170000<>2002151817<>02/15/20 Miami
    TN0530000<>20200730871<>07/16/20 LENOIR CITY
    TN0020100<>2007100050<>07/10/20 SHELBYVILLE
    TN0470000<>2008241521<>08/24/20 KNOX
    TN0610000<>202000043<>01/02/20 DECATUR
    TN0470000<>2008301940<>08/30/20 POWELL
    TN0440000<>080720-13540<>08/07/20 COOKEVILLE
    TN0470000<>2008301972<>08/30/20 CORRYTON
    TN0170000<>2008122130<>08/12/20 Gum Flat
    TN0020100<>2009190034<>09/19/20 SHELBYVILLE
    TN0540100<>202000000000<>09/28/20 Athens 
    TN0470000<>2010060450<>10/06/20 POWELL
    TN0470000<>2008160966<>08/16/20 POWELL
    TN0440000<>090520-13613<>09/05/20 COOKEVILLE
    TN0190100<>20200588121<>09/12/20 Nashville
    TN0410000<>201022-1156<>10/22/20 NUNNELLY
    TN0470000<>2011141072<>11/14/20 MASCOT
    TN0470000<>2011201575<>11/20/20 STRAWBERRY PLAINS
    TN0470000<>2011241935<>11/24/20 KNOX
    TN0830100<>GLTN20-04602<>11/03/20 GALLATIN
    TN0470000<>2012241929<>12/24/20 KNOXVILLE
    TN0440000<>011620-13127<>01/16/20 GAINESBORO
    TN0440000<>072020-13497<>07/19/20 GAINESBORO
    TN0470000<>2012080552<>12/08/20 STRAWBERRY PLNS
    TN0470000<>2012120991<>12/12/20 CORRYTON
    TN0470000<>2102221624<>09/01/20 POWELL
    TN0470000<>2007181150<>07/18/20 KNOX
    TN0640000<>2020-0436<>12/12/20 FAYETTEVILLE
    TN0410000<>201031-1322<>10/31/20 NUNNELLY
    TN0170000<>2009041123<>09/04/20 Paragould
    TN0190100<>20200414761<>06/19/20 ANTIOCH
    TN0610000<>202005168<>07/01/20 BIRCHWOOD
    TN0740100<>2020-001624<>07/11/20 SPRINGFIELD
    TN0330200<>20-017126<>12/30/20 EAST RIDGE'

}

In [64]:
admin_df.loc[admin_df.Incident_Key == 'TN0470000<>2007120722<>07/12/20']

,ORI,Incident Number,Incident Date,Report Date Indicator,Hour of Day,Incident Clearance,Exceptional Clearance Date,Address,Cargo Theft,Incident_Key
22675,TN0470000,2007120722,07/12/20,NaN,12:00n-12:59pm,Not Cleared,NaN,"5209,ODELL RD,,KNOX,,",Missing,TN0470000<>2007120722<>07/12/20
